In [64]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [85]:
driver = webdriver.Chrome()

In [86]:
driver.get("https://cgmix.uscg.mil/IIR/IIRSearch.aspx")

In [87]:
begin_date_xpath = '//*[@id="TextBoxFromDate"]'

begin_date = driver.find_element(By.XPATH, begin_date_xpath)

end_date_xpath = '//*[@id="TextBoxToDate"]'
end_date = driver.find_element(By.XPATH, end_date_xpath)

In [88]:
vessel_service_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[4]/td[2]/select"
vessel_service = driver.find_element(By.XPATH, vessel_service_xpath)

involved_org_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[6]/td[2]/input"
involved_org = driver.find_element(By.XPATH, involved_org_xpath)

In [89]:
kword_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[8]/td[2]/input"

kword = driver.find_element(By.XPATH, kword_xpath)

search_btn_xpath = '/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[13]/td/input[1]'
search_btn = driver.find_element(By.XPATH, search_btn_xpath)

In [90]:
def set_begin_date(date):
    begin_date.clear()
    begin_date.send_keys(date)

def set_end_date(date):
    end_date.clear()
    end_date.send_keys(date)

def set_involved_org(org):
    involved_org.clear()
    involved_org.send_keys(org)

def set_vessel_service(service):
    vessel_service.send_keys(service)

def set_kword(kw):
    kword.clear()
    kword.send_keys(kw)

In [91]:
set_begin_date("01/01/2020")
set_vessel_service("Freight Ship")
search_btn.click()

In [95]:
table_xpath = '/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/div/table/tbody/tr[5]/td/div/table/tbody'
table = driver.find_element(By.XPATH, table_xpath)

trs = table.find_elements(By.TAG_NAME, 'tr')

In [92]:
def gen_links(trs):
    details_links = []
    for i, tr in enumerate(trs[1:]):
        try:
            tds = tr.find_elements(By.TAG_NAME, 'td')
            details_link = tds[0].find_element(By.TAG_NAME, 'a')
            title = tds[1].text
            details_links.append((title, details_link))
        except:
            print("failed on " + str(i))
            details_links.append(tds)
    return details_links

In [96]:
gen_links(trs)

failed on 11


[('ALGOMA BUFFALO - EMERG SHUTDOWN - ANCHORAGE',
  <selenium.webdriver.remote.webelement.WebElement (session="b9666b48edcb1ad06fe324ed4b0e4d82", element="f.76DB370F488BF551E574D80E1714E4E2.d.AF80462EA16A5826FBBE778B45000AE2.e.77")>),
 ('MS-Loss of Propulsion/MV MELOI/City Dock #26',
  <selenium.webdriver.remote.webelement.WebElement (session="b9666b48edcb1ad06fe324ed4b0e4d82", element="f.76DB370F488BF551E574D80E1714E4E2.d.AF80462EA16A5826FBBE778B45000AE2.e.82")>),
 ('WALTER J. MCCARTHY - FIRE',
  <selenium.webdriver.remote.webelement.WebElement (session="b9666b48edcb1ad06fe324ed4b0e4d82", element="f.76DB370F488BF551E574D80E1714E4E2.d.AF80462EA16A5826FBBE778B45000AE2.e.87")>),
 ('SBI PISCES / LOSS OF RPMs - GROUNDING',
  <selenium.webdriver.remote.webelement.WebElement (session="b9666b48edcb1ad06fe324ed4b0e4d82", element="f.76DB370F488BF551E574D80E1714E4E2.d.AF80462EA16A5826FBBE778B45000AE2.e.92")>),
 ('CAPE HUDSON - Loss of Propulsion',
  <selenium.webdriver.remote.webelement.WebElemen

In [171]:
def refresh_links(driver):
    table_xpath = '/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/div/table/tbody/tr[5]/td/div/table/tbody'
    table = driver.find_element(By.XPATH, table_xpath)

    trs = table.find_elements(By.TAG_NAME, 'tr')
    details_links = gen_links(trs[:11])
    return details_links

In [74]:
def process_page(driver, qa=False):
    from time import sleep
    from random import randint
    sleep(randint(1, 3))
    div_xpath = "/html/body/form/div[3]/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div"
    div = driver.find_element(By.XPATH, div_xpath)
    tbls = div.find_elements(By.TAG_NAME, 'table')
    pg_tables =[]
    for tbl in tbls:
        # save tbl text to string
        s = tbl.text
        pg_tables.append(s)
        # print(tbl.text)
    if qa:
        # need to start at top of page
        driver.execute_script("window.scrollTo(0, 0)")
        # need to loop over length of page
        # can I get the length of the page from the driver object?
        ht = driver.execute_script("return document.body.scrollHeight")
        for i in range(0, ht, 100):
            driver.execute_script("window.scrollTo(0, " + str(i) + ")")
            driver.save_screenshot(str(i) +"_screenshot.png")
        
    # return to previous page
    driver.back()
    return pg_tables


In [202]:
import pickle
import tqdm
curr = 0
ctr = 1
c = 1
progress = tqdm.tqdm(1670)
while True:
    print("Scraping page " + str(ctr))
    pgs = []
    for i in range(10):
        links = refresh_links(driver)
        links[i][1].click()
        title = ["TITLE: "+links[i][0]]
        pg = process_page(driver)
        pgs.append(title + pg)
        progress.update(1)
    print("-"*5 + "saving page " + str(ctr) + "-"*5)
    with open("page_" + str(ctr) + ".pkl", "wb") as f:
        pickle.dump(pgs, f)
    ctr += 1
    pg_bar_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/div/table/tbody/tr[5]/td/div/table/tbody/tr[12]/td/table/tbody/tr"
    pg_bar = driver.find_element(By.XPATH, pg_bar_xpath)
    curr %= 11
    curr += 1
    nxt_pg = pg_bar.find_elements(By.TAG_NAME, "td")
    nxt_pg[curr].click()


10it [00:46,  4.69s/it]

-----saving page 1-----


20it [01:28,  4.01s/it]

-----saving page 1-----


30it [02:11,  4.03s/it]

...

40it [02:49,  3.64s/it]

-----saving page 1-----


50it [03:39,  4.33s/it]

-----saving page 1-----
failed on 9


53it [03:53,  4.66s/it]

In [143]:
from functools import reduce
t1 = pgs[0][0]
p1 = map(lambda x: x.replace("\n", "").replace("'", "").replace("\r","").strip(), pgs[0][1:])
p1 = "\n".join(p1)

('Investigation Activity ReportALGOMA BUFFALO - EMERG SHUTDOWN - '
 'ANCHORAGEActivity Start Date: Thursday, January 2, 2020MISLE Activity '
 'Number: 6881556MISLE Originating Unit: Sector DetroitMISLE Activity Owner: '
 'Commandant (CG-INV-3)MISLE Activity Controller:MISLE Case Number: Not '
 'Asscoiated with a CaseI. INCIDENT BRIEFThe ALGOMA BUFFALO sustained an '
 'equipment failure of a hydraulic line which caused fluid to spray on the '
 'port main engine and caused smoke. The Master shut down the main engines and '
 'went to emergency anchorage as a precaution. Because of the emergency '
 'shutdown and loss of vessel maneuverability the casualty will be '
 'investigated at the informal level.\n'
 'II. INCIDENT SUMMARYIncident Involved: Marine Casualty, ReportableLevel of '
 'Investigation: InformalIMO Classification: RoutineUSCG Classification: '
 'RoutineWas This a Serious Marine Incident? NoWas a Marine Board Convened by '
 'Commandant? No\n'
 'Personal Casualty Summary Vessel(

In [129]:
pgs[0][1].replace("\n", "")

'Investigation Activity ReportALGOMA BUFFALO - EMERG SHUTDOWN - ANCHORAGEActivity Start Date: Thursday, January 2, 2020MISLE Activity Number: 6881556MISLE Originating Unit: Sector DetroitMISLE Activity Owner: Commandant (CG-INV-3)MISLE Activity Controller:MISLE Case Number: Not Asscoiated with a CaseI. INCIDENT BRIEFThe ALGOMA BUFFALO sustained an equipment failure of a hydraulic line which caused fluid to spray on the port main engine and caused smoke. The Master shut down the main engines and went to emergency anchorage as a precaution. Because of the emergency shutdown and loss of vessel maneuverability the casualty will be investigated at the informal level.'

In [76]:
import pickle as pk
pk.dumps(pgs)
with open('pgs.pkl', 'wb') as f:
    pk.dump(pgs, f)

In [77]:
with open('pgs.pkl', "rb") as f:
    tst = pk.load(f)

In [84]:
tst[0]

['Investigation Activity Report\nALGOMA BUFFALO - EMERG SHUTDOWN - ANCHORAGE\nActivity Start Date: Thursday, January 2, 2020\nMISLE Activity Number: 6881556\nMISLE Originating Unit: Sector Detroit\nMISLE Activity Owner: Commandant (CG-INV-3)\nMISLE Activity Controller:\nMISLE Case Number: Not Asscoiated with a Case\nI. INCIDENT BRIEF\nThe ALGOMA BUFFALO sustained an equipment failure of a hydraulic line which caused fluid to spray on the port main engine and caused smoke. The Master shut down the main engines and went to emergency anchorage as a precaution. Because of the emergency shutdown and loss of vessel maneuverability the casualty will be investigated at the informal level.',
 'II. INCIDENT SUMMARY\nIncident Involved: Marine Casualty, Reportable\nLevel of Investigation: Informal\nIMO Classification: Routine\nUSCG Classification: Routine\nWas This a Serious Marine Incident? No\nWas a Marine Board Convened by Commandant? No',
 'Personal Casualty Summary Vessel(s) Status Summary Pr

In [28]:
# TODO :: regenerate links each time driver returns to search results .. :(